In [1]:
%%bash
chia version
cdv --version
python --version

1.6.1b4.dev15
cdv, version 1.1.2
Python 3.10.7


In [2]:
# chia libraries
from blspy import (PrivateKey, AugSchemeMPL, G2Element)

from chia.consensus.default_constants import DEFAULT_CONSTANTS
from chia.types.blockchain_format.coin import Coin
from chia.types.blockchain_format.program import Program
from chia.types.coin_spend import CoinSpend
from chia.types.condition_opcodes import ConditionOpcode
from chia.types.spend_bundle import SpendBundle
from chia.util.hash import std_hash
from chia.wallet.puzzles import p2_delegated_puzzle_or_hidden_puzzle

from clvm.casts import int_to_bytes
from clvm_tools.clvmc import compile_clvm_text
from clvm_tools.binutils import disassemble

# utils & tic tac toe helper code
import sys
sys.path.insert(0, "../code")
from utils import (load_program, print_program, print_puzzle, print_json, print_push_tx_result)
import singleton_utils
import tic_tac_toe

# puzzles
tic_tac_toe_puzzle = load_program("../code/tic-tac-toe.clsp", ["../code"])
coin_puzzle = load_program("../code/coin.clsp", ["../code"])
terminate_puzzle = load_program("../code/terminate-game.clsp", ["../code"])

waiting_room_puzzle = load_program("../code/waiting-room.clsp", ["../code"])
play_amount = 1_000_000_000_000
game_amount = (play_amount * 2) + 1 # singleton odd mojos

## 1. Set Up A Blockchain Simulator

In [3]:
from cdv.test import Network, Wallet

network: Network = await Network.create()

# use as function parameters
get_coin_records_by_parent_ids = network.sim_client.get_coin_records_by_parent_ids
get_coin_records_by_puzzle_hash = network.sim_client.get_coin_records_by_puzzle_hash
get_coin_record_by_name = network.sim_client.get_coin_record_by_name
get_puzzle_and_solution = network.sim_client.get_puzzle_and_solution        
get_block_records = network.sim_client.get_block_records
get_additions_and_removals = network.sim_client.get_additions_and_removals

await network.farm_block()

alice: Wallet = network.make_wallet("alice")
bob: Wallet = network.make_wallet("bob")
await network.farm_block(farmer=alice)
await network.farm_block(farmer=bob)

print(f'alice balance:\t\t{alice.balance()}')
print(f'alice pk:\t\t{alice.pk()}')
print(f'alice puzzle hash:\t{alice.puzzle_hash}')
print(f'bob balance:\t\t{bob.balance()}')
print(f'alice pk:\t\t{alice.pk()}')
print(f'bob puzzle hash:\t{bob.puzzle_hash}')

# prepare players info
player_one_info = Program.to([alice.pk(), alice.puzzle_hash])
player_two_info = Program.to([bob.pk(), bob.puzzle_hash])

alice_sk = alice.sk_
alice_pk = alice.pk()
alice_puzzle_hash = alice.puzzle_hash
bob_sk = bob.sk_
bob_pk = bob.pk()
bob_puzzle_hash = bob.puzzle_hash

alice balance:		2000000000000
alice pk:		aba7ed288dd79189bec34698a3437fa7a45f801596d397a4f70081a0956dcdbe998388bfa758f4d49fa421ce3850a6d8
alice puzzle hash:	4f45877796d7a64e192bcc9f899afeedae391f71af3afd7e15a0792c049d23d3
bob balance:		2000000000000
alice pk:		aba7ed288dd79189bec34698a3437fa7a45f801596d397a4f70081a0956dcdbe998388bfa758f4d49fa421ce3850a6d8
bob puzzle hash:	87908e3f85bf4b55c7e7709915c2ce97a1e6ec1d227e54a04dbfee6862d546a5


## 2. Create Waiting Room Coins
[waiting-room.clsp](../code/waiting-room.clsp)

In [4]:
from chia.util.bech32m import encode_puzzle_hash
## alice
alice_waiting_room_puzzle = waiting_room_puzzle.curry(
    alice_puzzle_hash,
    alice_pk, 
    bob_pk,
    0, # P1_COIN_ID is null
    play_amount,
    game_amount
)
alice_waiting_room_puzzle_hash = alice_waiting_room_puzzle.get_tree_hash()
print(alice_waiting_room_puzzle_hash.hex())
print_program(alice_waiting_room_puzzle)

e690bf9dea6880c75cd64a7ff22fefa727597190cdbbee2663e0af7e1b5d129b
(a (q 2 (q 2 (i 383 (q 2 62 (c 2 (c 11 (c 23 (c 47 (c -65 (c 383 (c 767 ())))))))) (q 4 (c 20 (c 44 ())) (c (c 60 (c 5 (c 95 ()))) (c (c 16 (c (a (i 47 (q . 23) (q . 11)) 1) (c (sha256 5 95) ()))) ())))) 1) (c (q ((50 . 61) 82 100 . 51) (60 . 0xeff07522495060c066f66f32acc2a77e3a3e737aca8baea4d1a64ea4cdc13da9) (a 46 (c 2 (c (c 5 (c 11 (c (c (q "game" . "tic-tac-toe") (c (c (q . "p1_pk") 23) (c (c (q . "p2_pk") 47) ()))) ()))) ()))) (a (i (l 5) (q 11 (q . 2) (a 46 (c 2 (c 9 ()))) (a 46 (c 2 (c 13 ())))) (q 11 (q . 1) 5)) 1) 2 (i 23 (q 4 (c 24 (c (sha256 23 47) ())) ()) (q 4 (c 24 (c (sha256 95 (a 22 (c 2 (c -65 (c 47 (c 5 (c 11 ()))))))) ())) (c (c 60 (c 26 (c 47 ()))) (c (c 18 (c 47 ())) (c (c 16 (c 5 (c (sha256 95 -65) ()))) ()))))) 1) 1)) (c (q . 0x4f45877796d7a64e192bcc9f899afeedae391f71af3afd7e15a0792c049d23d3) (c (q . 0xaba7ed288dd79189bec34698a3437fa7a45f801596d397a4f70081a0956dcdbe998388bfa758f4d49fa421ce3850a6d8) (

In [5]:
# alice's coin creates a waiting room coin
alice_coin_wrapper = await alice.choose_coin(1_750_000_000_000)
alice_conditions = [
    [
        ConditionOpcode.CREATE_COIN,
        alice_waiting_room_puzzle_hash,
        play_amount + 1
    ],
    
    # change
    [
        ConditionOpcode.CREATE_COIN, 
        alice.puzzle_hash, 
        alice_coin_wrapper.coin.amount - (play_amount + 1)
    ]
]

coin_spend, signature = alice_coin_wrapper.create_standard_spend(alice_sk, alice_conditions)
print_json(coin_spend.to_json_dict())
print(signature)

{
    "coin": {
        "amount": 1750000000000,
        "parent_coin_info": "0xe3b0c44298fc1c149afbf4c8996fb92400000000000000000000000000000001",
        "puzzle_hash": "0x4f45877796d7a64e192bcc9f899afeedae391f71af3afd7e15a0792c049d23d3"
    },
    "puzzle_reveal": "0xff02ffff01ff02ffff01ff02ffff03ff0bffff01ff02ffff03ffff09ff05ffff1dff0bffff1effff0bff0bffff02ff06ffff04ff02ffff04ff17ff8080808080808080ffff01ff02ff17ff2f80ffff01ff088080ff0180ffff01ff04ffff04ff04ffff04ff05ffff04ffff02ff06ffff04ff02ffff04ff17ff80808080ff80808080ffff02ff17ff2f808080ff0180ffff04ffff01ff32ff02ffff03ffff07ff0580ffff01ff0bffff0102ffff02ff06ffff04ff02ffff04ff09ff80808080ffff02ff06ffff04ff02ffff04ff0dff8080808080ffff01ff0bffff0101ff058080ff0180ff018080ffff04ffff01b0a042c855d234578415254b7870b711fb25e8f85beaa4a66bd0673d394c761fa156406c2e3bb375d5b18766d2a12cc918ff018080",
    "solution": "0xff80ffff01ffff33ffa0e690bf9dea6880c75cd64a7ff22fefa727597190cdbbee2663e0af7e1b5d129bff8600e8d4a5100180ffff33ffa04f45877796d7a6

In [6]:
from chia.types.spend_bundle import SpendBundle
spend_bundle = SpendBundle(
    [ coin_spend],
    signature
)
result = await network.push_tx(spend_bundle)
print_push_tx_result(result)

additions:
Coin { parent_coin_info: 12d7b8c1654f82f2330059abc28e3240e863450706de7fdc518026f393f68bba, puzzle_hash: 4f45877796d7a64e192bcc9f899afeedae391f71af3afd7e15a0792c049d23d3, amount: 749999999999 }
Coin { parent_coin_info: 12d7b8c1654f82f2330059abc28e3240e863450706de7fdc518026f393f68bba, puzzle_hash: e690bf9dea6880c75cd64a7ff22fefa727597190cdbbee2663e0af7e1b5d129b, amount: 1000000000001 }
removals:
Coin { parent_coin_info: e3b0c44298fc1c149afbf4c8996fb92400000000000000000000000000000001, puzzle_hash: 4f45877796d7a64e192bcc9f899afeedae391f71af3afd7e15a0792c049d23d3, amount: 1750000000000 }


In [7]:
coin_records = await get_coin_records_by_puzzle_hash(alice_waiting_room_puzzle_hash)
alice_waiting_room_coin = coin_records[0].coin
print_json(alice_waiting_room_coin.to_json_dict())
print(alice_waiting_room_coin.name().hex())

{
    "amount": 1000000000001,
    "parent_coin_info": "0x12d7b8c1654f82f2330059abc28e3240e863450706de7fdc518026f393f68bba",
    "puzzle_hash": "0xe690bf9dea6880c75cd64a7ff22fefa727597190cdbbee2663e0af7e1b5d129b"
}
6b95688d081271b4f2466a3fd0301cefb62c33712d09e6a0b43e05257eb3ad52


In [8]:
## bob
bob_waiting_room_puzzle = waiting_room_puzzle.curry(
    bob_puzzle_hash,
    alice_pk, 
    bob_pk,
    alice_waiting_room_coin.name(),
    play_amount,
    game_amount
    
)
bob_waiting_room_puzzle_hash = bob_waiting_room_puzzle.get_tree_hash()
print(bob_waiting_room_puzzle_hash.hex())
print_program(bob_waiting_room_puzzle)

0fdc47b8db534c7168295bb5f21235aff4410e16173ac1467b086ee8cac58ba0
(a (q 2 (q 2 (i 383 (q 2 62 (c 2 (c 11 (c 23 (c 47 (c -65 (c 383 (c 767 ())))))))) (q 4 (c 20 (c 44 ())) (c (c 60 (c 5 (c 95 ()))) (c (c 16 (c (a (i 47 (q . 23) (q . 11)) 1) (c (sha256 5 95) ()))) ())))) 1) (c (q ((50 . 61) 82 100 . 51) (60 . 0xeff07522495060c066f66f32acc2a77e3a3e737aca8baea4d1a64ea4cdc13da9) (a 46 (c 2 (c (c 5 (c 11 (c (c (q "game" . "tic-tac-toe") (c (c (q . "p1_pk") 23) (c (c (q . "p2_pk") 47) ()))) ()))) ()))) (a (i (l 5) (q 11 (q . 2) (a 46 (c 2 (c 9 ()))) (a 46 (c 2 (c 13 ())))) (q 11 (q . 1) 5)) 1) 2 (i 23 (q 4 (c 24 (c (sha256 23 47) ())) ()) (q 4 (c 24 (c (sha256 95 (a 22 (c 2 (c -65 (c 47 (c 5 (c 11 ()))))))) ())) (c (c 60 (c 26 (c 47 ()))) (c (c 18 (c 47 ())) (c (c 16 (c 5 (c (sha256 95 -65) ()))) ()))))) 1) 1)) (c (q . 0x87908e3f85bf4b55c7e7709915c2ce97a1e6ec1d227e54a04dbfee6862d546a5) (c (q . 0xaba7ed288dd79189bec34698a3437fa7a45f801596d397a4f70081a0956dcdbe998388bfa758f4d49fa421ce3850a6d8) (

In [9]:
# bob's coin creates a waiting room coin
bob_coin_wrapper = await bob.choose_coin(1_750_000_000_000)
bob_conditions = [
    [
        ConditionOpcode.CREATE_COIN,
        bob_waiting_room_puzzle_hash,
        play_amount
    ],
    
    # change
    [
        ConditionOpcode.CREATE_COIN, 
        bob.puzzle_hash, 
        bob_coin_wrapper.coin.amount - (play_amount)
    ]
]

coin_spend, signature = bob_coin_wrapper.create_standard_spend(bob_sk, bob_conditions)
print_json(coin_spend.to_json_dict())
print(signature)

{
    "coin": {
        "amount": 1750000000000,
        "parent_coin_info": "0xe3b0c44298fc1c149afbf4c8996fb92400000000000000000000000000000002",
        "puzzle_hash": "0x87908e3f85bf4b55c7e7709915c2ce97a1e6ec1d227e54a04dbfee6862d546a5"
    },
    "puzzle_reveal": "0xff02ffff01ff02ffff01ff02ffff03ff0bffff01ff02ffff03ffff09ff05ffff1dff0bffff1effff0bff0bffff02ff06ffff04ff02ffff04ff17ff8080808080808080ffff01ff02ff17ff2f80ffff01ff088080ff0180ffff01ff04ffff04ff04ffff04ff05ffff04ffff02ff06ffff04ff02ffff04ff17ff80808080ff80808080ffff02ff17ff2f808080ff0180ffff04ffff01ff32ff02ffff03ffff07ff0580ffff01ff0bffff0102ffff02ff06ffff04ff02ffff04ff09ff80808080ffff02ff06ffff04ff02ffff04ff0dff8080808080ffff01ff0bffff0101ff058080ff0180ff018080ffff04ffff01b0ad23d290a6569d41be4f58704709812f466cc10c5720fee77bf491d0fae2164a6eb57e2c4158879d9e25f18ce515a306ff018080",
    "solution": "0xff80ffff01ffff33ffa00fdc47b8db534c7168295bb5f21235aff4410e16173ac1467b086ee8cac58ba0ff8600e8d4a5100080ffff33ffa087908e3f85bf4b

In [10]:
from chia.types.spend_bundle import SpendBundle
spend_bundle = SpendBundle(
    [ coin_spend],
    signature
)
result = await network.push_tx(spend_bundle)
print_push_tx_result(result)

additions:
Coin { parent_coin_info: 17ca02c0a209d7e1a3869442ba13ef9468181c4b095b8823aeaf3c27f8e58c34, puzzle_hash: 0fdc47b8db534c7168295bb5f21235aff4410e16173ac1467b086ee8cac58ba0, amount: 1000000000000 }
Coin { parent_coin_info: 17ca02c0a209d7e1a3869442ba13ef9468181c4b095b8823aeaf3c27f8e58c34, puzzle_hash: 87908e3f85bf4b55c7e7709915c2ce97a1e6ec1d227e54a04dbfee6862d546a5, amount: 750000000000 }
removals:
Coin { parent_coin_info: e3b0c44298fc1c149afbf4c8996fb92400000000000000000000000000000002, puzzle_hash: 87908e3f85bf4b55c7e7709915c2ce97a1e6ec1d227e54a04dbfee6862d546a5, amount: 1750000000000 }


## 3. Create Singleton Game Coin

### 3.1 Terminate Puzzle (Curried)
[terminate-game.clsp](../code/terminate-game.clsp)

In [11]:
player_fee = 50_000_000
p2_amount = play_amount - player_fee
curried_terminate_puzzle = terminate_puzzle.curry(
    1, # Terminate Singleton Coin
    alice_puzzle_hash,
    bob_puzzle_hash,
    p2_amount
)

### 3.2 Coin Puzzle (Curried)
[coin.clsp](../code/coin.clsp)

In [12]:
def get_coin_puzzle(board, player):
    # (mod (BOARD V pos)
    curried_tic_tac_toe_puzzle = tic_tac_toe_puzzle.curry(
            Program.to(board), 
            Program.to(player)
        ) 

    #(mod (MOD PLAYER_ONE_INFO PLAYER_TWO_INFO CURRIED_TIC_TAC_TOE_PUZZLE amount position)
    curried_coin_puzzle = coin_puzzle.curry(
        coin_puzzle,
        curried_terminate_puzzle,
        player_one_info,
        player_two_info,
        curried_tic_tac_toe_puzzle,
        game_amount)
    return curried_coin_puzzle

curried_coin_puzzle = get_coin_puzzle([' '] * 9, 'x')

### 3.3 Launcher Coin

In [13]:
coin_records = await get_coin_records_by_puzzle_hash(alice_waiting_room_puzzle_hash)
alice_waiting_room_coin = coin_records[0].coin
coin_records = await get_coin_records_by_puzzle_hash(bob_waiting_room_puzzle_hash)
bob_waiting_room_coin = coin_records[0].coin

In [14]:
from chia.types.blockchain_format.coin import Coin
from chia.types.coin_spend import CoinSpend
from chia.wallet.puzzles import singleton_top_layer_v1_1

# prepare launcher coin
launcher_coin = Coin(
    alice_waiting_room_coin.name(), # waiting_room's coin spend creates the launcher coin
    singleton_top_layer_v1_1.SINGLETON_LAUNCHER_HASH, 
    game_amount
)

launcher_id = launcher_coin.name()
print(f'\nlauncher id: {launcher_id.hex()}')

singleton_struct = (
    singleton_top_layer_v1_1.SINGLETON_MOD_HASH, 
    (launcher_id, singleton_top_layer_v1_1.SINGLETON_LAUNCHER_HASH)
)

singleton_puzzle = singleton_top_layer_v1_1.SINGLETON_MOD.curry(
    singleton_struct,
    curried_coin_puzzle, # tic tac toe coin puzzle
)
print_json(launcher_coin.to_json_dict())


launcher id: 24658905e1bde4e544d5322ae6629803307d640133dc3c382854f8d62e2fbb62
{
    "amount": 2000000000001,
    "parent_coin_info": "0x6b95688d081271b4f2466a3fd0301cefb62c33712d09e6a0b43e05257eb3ad52",
    "puzzle_hash": "0xeff07522495060c066f66f32acc2a77e3a3e737aca8baea4d1a64ea4cdc13da9"
}


### 3.4 Singleton Launcher
[singleton_top_layer.clvm](https://github.com/Chia-Network/chia-blockchain/blob/main/chia/wallet/puzzles/singleton_top_layer.clvm)

[singleton_launcher.clvm](https://github.com/Chia-Network/chia-blockchain/blob/main/chia/wallet/puzzles/singleton_launcher.clvm)

In [15]:
launcher_solution = Program.to(
    [
        singleton_puzzle.get_tree_hash(),
        game_amount,
        [
            ("game", "tic-tac-toe"), 
            ("p1_pk", alice_pk),
            ("p2_pk", bob_pk)
        ]
    ]
)
print_program(launcher_solution)

launcher_announcement = launcher_solution.get_tree_hash()
print(f'\nlauncher_announcement:\n{launcher_announcement}')

launcher_coin_spend = CoinSpend(
    launcher_coin,
    singleton_top_layer_v1_1.SINGLETON_LAUNCHER,
    launcher_solution
)
print_json(launcher_coin_spend.to_json_dict())

(0xe9c900365fd7fe593816f6b7bb2e7bba4b04e65fc45bf0e4a5576dba96df3bce 0x01d1a94a2001 (("game" . "tic-tac-toe") ("p1_pk" . 0xaba7ed288dd79189bec34698a3437fa7a45f801596d397a4f70081a0956dcdbe998388bfa758f4d49fa421ce3850a6d8) ("p2_pk" . 0xb01da35fe872b5c9c342d2a3ac6dfbe800eaf35c687a512a172d329f3bd10a83e675018a90a36178c81c0687b49d167f)))

launcher_announcement:
c8712fe4b52f02748923af8bf3a2ecd2ea65bc27a08eaa44bccfdf21a62e5d5e
{
    "coin": {
        "amount": 2000000000001,
        "parent_coin_info": "0x6b95688d081271b4f2466a3fd0301cefb62c33712d09e6a0b43e05257eb3ad52",
        "puzzle_hash": "0xeff07522495060c066f66f32acc2a77e3a3e737aca8baea4d1a64ea4cdc13da9"
    },
    "puzzle_reveal": "0xff02ffff01ff04ffff04ff04ffff04ff05ffff04ff0bff80808080ffff04ffff04ff0affff04ffff02ff0effff04ff02ffff04ffff04ff05ffff04ff0bffff04ff17ff80808080ff80808080ff808080ff808080ffff04ffff01ff33ff3cff02ffff03ffff07ff0580ffff01ff0bffff0102ffff02ff0effff04ff02ffff04ff09ff80808080ffff02ff0effff04ff02ffff04ff0dff80808080

In [16]:
print(f'launcher_announcement:\n{launcher_announcement}')
print(f'\nlauncher_coin_id:\n{launcher_id.hex()}')

# announcementID == sha256(coinID + message).
from chia.util.hash import std_hash
launcher_coin_announcement = std_hash(launcher_id + launcher_announcement)
print(f'\nlauncher_coin_announcement:\n{launcher_coin_announcement}')

launcher_announcement:
c8712fe4b52f02748923af8bf3a2ecd2ea65bc27a08eaa44bccfdf21a62e5d5e

launcher_coin_id:
24658905e1bde4e544d5322ae6629803307d640133dc3c382854f8d62e2fbb62

launcher_coin_announcement:
df929e020f10443d08029d4703585e26a1e6471d5defdff39680ec228b48e553


### 3.5 Waiting Room Coins
[waiting-room.clsp](../code/waiting-room.clsp)

In [17]:
alice_waiting_room_coin_spend = CoinSpend(
    alice_waiting_room_coin,
    alice_waiting_room_puzzle,
    Program.to([
        launcher_id,
        singleton_puzzle.get_tree_hash()
    ])
)
print_json(alice_waiting_room_coin_spend.to_json_dict())

{
    "coin": {
        "amount": 1000000000001,
        "parent_coin_info": "0x12d7b8c1654f82f2330059abc28e3240e863450706de7fdc518026f393f68bba",
        "puzzle_hash": "0xe690bf9dea6880c75cd64a7ff22fefa727597190cdbbee2663e0af7e1b5d129b"
    },
    "puzzle_reveal": "0xff02ffff01ff02ffff01ff02ffff03ff82017fffff01ff02ff3effff04ff02ffff04ff0bffff04ff17ffff04ff2fffff04ff81bfffff04ff82017fffff04ff8202ffff808080808080808080ffff01ff04ffff04ff14ffff04ff2cff808080ffff04ffff04ff3cffff04ff05ffff04ff5fff80808080ffff04ffff04ff10ffff04ffff02ffff03ff2fffff0117ffff010b80ff0180ffff04ffff0bff05ff5f80ff80808080ff8080808080ff0180ffff04ffff01ffffff323dff52ff6433ffff3ca0eff07522495060c066f66f32acc2a77e3a3e737aca8baea4d1a64ea4cdc13da9ffff02ff2effff04ff02ffff04ffff04ff05ffff04ff0bffff04ffff04ffff01ff8467616d658b7469632d7461632d746f65ffff04ffff04ffff018570315f706bff1780ffff04ffff04ffff018570325f706bff2f80ff80808080ff80808080ff80808080ffff02ffff03ffff07ff0580ffff01ff0bffff0102ffff02ff2effff04ff02ffff04ff09ff80

In [18]:
bob_waiting_room_coin_spend = CoinSpend(
    bob_waiting_room_coin,
    bob_waiting_room_puzzle,
    Program.to([
        launcher_id,
        singleton_puzzle.get_tree_hash()
    ])
)
print_json(bob_waiting_room_coin_spend.to_json_dict())

{
    "coin": {
        "amount": 1000000000000,
        "parent_coin_info": "0x17ca02c0a209d7e1a3869442ba13ef9468181c4b095b8823aeaf3c27f8e58c34",
        "puzzle_hash": "0x0fdc47b8db534c7168295bb5f21235aff4410e16173ac1467b086ee8cac58ba0"
    },
    "puzzle_reveal": "0xff02ffff01ff02ffff01ff02ffff03ff82017fffff01ff02ff3effff04ff02ffff04ff0bffff04ff17ffff04ff2fffff04ff81bfffff04ff82017fffff04ff8202ffff808080808080808080ffff01ff04ffff04ff14ffff04ff2cff808080ffff04ffff04ff3cffff04ff05ffff04ff5fff80808080ffff04ffff04ff10ffff04ffff02ffff03ff2fffff0117ffff010b80ff0180ffff04ffff0bff05ff5f80ff80808080ff8080808080ff0180ffff04ffff01ffffff323dff52ff6433ffff3ca0eff07522495060c066f66f32acc2a77e3a3e737aca8baea4d1a64ea4cdc13da9ffff02ff2effff04ff02ffff04ffff04ff05ffff04ff0bffff04ffff04ffff01ff8467616d658b7469632d7461632d746f65ffff04ffff04ffff018570315f706bff1780ffff04ffff04ffff018570325f706bff2f80ff80808080ff80808080ff80808080ffff02ffff03ffff07ff0580ffff01ff0bffff0102ffff02ff2effff04ff02ffff04ff09ff80

In [19]:
from chia.types.spend_bundle import SpendBundle
from chia.util.hash import std_hash

message: bytes = std_hash(launcher_id + singleton_puzzle.get_tree_hash())
sig_alice_spend: G2Element = AugSchemeMPL.sign(
    alice_sk,
    message
    + alice_waiting_room_coin.name()
    + DEFAULT_CONSTANTS.AGG_SIG_ME_ADDITIONAL_DATA
)
spend_bundle = SpendBundle([
        launcher_coin_spend,
        alice_waiting_room_coin_spend,
        bob_waiting_room_coin_spend
    ],
    sig_alice_spend
)
print_json(spend_bundle.to_json_dict(include_legacy_keys = False, exclude_modern_keys = False))

{
    "aggregated_signature": "0x80cc516f4fe82c5fbe49798a3ec27d261d187c5baf0e988de3f85f85a0e6693fe1e18ea96b14de856584798599657a9d08bd93bc9db7f970598b8eff689e9bdae4ede503108641b0228cce54c33bb268f52b1ea907513dfa5580c608bfa0d6d1",
    "coin_spends": [
        {
            "coin": {
                "amount": 2000000000001,
                "parent_coin_info": "0x6b95688d081271b4f2466a3fd0301cefb62c33712d09e6a0b43e05257eb3ad52",
                "puzzle_hash": "0xeff07522495060c066f66f32acc2a77e3a3e737aca8baea4d1a64ea4cdc13da9"
            },
            "puzzle_reveal": "0xff02ffff01ff04ffff04ff04ffff04ff05ffff04ff0bff80808080ffff04ffff04ff0affff04ffff02ff0effff04ff02ffff04ffff04ff05ffff04ff0bffff04ff17ff80808080ff80808080ff808080ff808080ffff04ffff01ff33ff3cff02ffff03ffff07ff0580ffff01ff0bffff0102ffff02ff0effff04ff02ffff04ff09ff80808080ffff02ff0effff04ff02ffff04ff0dff8080808080ffff01ff0bffff0101ff058080ff0180ff018080",
            "solution": "0xffa0e9c900365fd7fe593816f6b7bb2e7bba4b04e65fc

### 3.6 Spend Waiting Room & Launcher Coins

In [20]:
result = await network.push_tx(spend_bundle)
print_push_tx_result(result)

additions:
Coin { parent_coin_info: 24658905e1bde4e544d5322ae6629803307d640133dc3c382854f8d62e2fbb62, puzzle_hash: e9c900365fd7fe593816f6b7bb2e7bba4b04e65fc45bf0e4a5576dba96df3bce, amount: 2000000000001 }
Coin { parent_coin_info: 6b95688d081271b4f2466a3fd0301cefb62c33712d09e6a0b43e05257eb3ad52, puzzle_hash: eff07522495060c066f66f32acc2a77e3a3e737aca8baea4d1a64ea4cdc13da9, amount: 2000000000001 }
removals:
Coin { parent_coin_info: 6b95688d081271b4f2466a3fd0301cefb62c33712d09e6a0b43e05257eb3ad52, puzzle_hash: eff07522495060c066f66f32acc2a77e3a3e737aca8baea4d1a64ea4cdc13da9, amount: 2000000000001 }
Coin { parent_coin_info: 12d7b8c1654f82f2330059abc28e3240e863450706de7fdc518026f393f68bba, puzzle_hash: e690bf9dea6880c75cd64a7ff22fefa727597190cdbbee2663e0af7e1b5d129b, amount: 1000000000001 }
Coin { parent_coin_info: 17ca02c0a209d7e1a3869442ba13ef9468181c4b095b8823aeaf3c27f8e58c34, puzzle_hash: 0fdc47b8db534c7168295bb5f21235aff4410e16173ac1467b086ee8cac58ba0, amount: 1000000000000 }


In [21]:
async def sim_play(prev_coin_spend, sk, board, player, position):
    network.sim.pass_blocks(1)

    lineage_proof = singleton_top_layer_v1_1.lineage_proof_for_coinsol(prev_coin_spend)
 
    curried_coin_puzzle = get_coin_puzzle(board, player)
    singleton_struct = (
        singleton_top_layer_v1_1.SINGLETON_MOD_HASH, 
        (launcher_id, singleton_top_layer_v1_1.SINGLETON_LAUNCHER_HASH)
    )

    singleton_puzzle = singleton_top_layer_v1_1.SINGLETON_MOD.curry(
                singleton_struct,
                curried_coin_puzzle,
    )

    singleton_coin = await singleton_utils.get_unspent_singleton(
        get_coin_records_by_parent_ids, 
        launcher_id)
    
    curried_tic_tac_toe_puzzle = tic_tac_toe.get_curried_puzzle_from_curried_coin_puzzle(curried_coin_puzzle)
    board_state, new_board = tic_tac_toe.play(curried_tic_tac_toe_puzzle, position)
    tic_tac_toe.print_board(new_board)

    coin_message = (
        std_hash(int_to_bytes(position))
        + singleton_coin.name()
        + DEFAULT_CONSTANTS.AGG_SIG_ME_ADDITIONAL_DATA
    )

    # sign with sk
    signature: G2Element = AugSchemeMPL.sign(
        sk,
        coin_message
    )
    
    inner_solution = Program.to([position]) # position
    coin_spend = singleton_utils.get_singleton_coin_spend(
        singleton_coin, singleton_puzzle, lineage_proof,
        inner_solution
    )

    #print(f'singleton_coin:\n{singleton_coin}')
    #print(f'\nlineage_proof:\n{lineage_proof}')
    #print(f'\ninner_solution:\n{inner_solution}')
    #print(f'\nsingleton_puzzle:\n{singleton_puzzle.get_tree_hash()}')

    #print('\ncoin_spend:')
    #print_json(coin_spend.to_json_dict())

    spend_bundle = SpendBundle([coin_spend], signature)
    #print_json(spend_bundle.to_json_dict())
    result = await network.push_tx(spend_bundle)
    print_push_tx_result(result)
    next_player = 'x' if player == 'o' else 'o'
    return new_board, next_player, coin_spend

## 4. Simulate Winning Board
1. Start with empty board.
2. Alice (`x`) plays first.
3. Bob (`o`) and Alice (`x`) take turn until Alice wins.

In [22]:
board = [' '] * 9
player = 'x'
board, player, coin_spend = await sim_play(launcher_coin_spend, alice.sk_, board, player, 4)
board, player, coin_spend = await sim_play(coin_spend, bob.sk_, board, player, 0)
board, player, coin_spend = await sim_play(coin_spend, alice.sk_, board, player, 3)
board, player, coin_spend = await sim_play(coin_spend, bob.sk_, board, player, 1)
board, player, coin_spend = await sim_play(coin_spend, alice.sk_, board, player, 5)

   |   |   
---+---+---
   | x |   
---+---+---
   |   |   

additions:
Coin { parent_coin_info: 79b7aaa318044a80ced7891fa6b8a0cc70e829ce73f04b8db11c29af7da50c67, puzzle_hash: 63d201d68ec302b4d15945c9a169fe397ab6aaf294b33f32c779f902dc295a14, amount: 2000000000001 }
removals:
Coin { parent_coin_info: 24658905e1bde4e544d5322ae6629803307d640133dc3c382854f8d62e2fbb62, puzzle_hash: e9c900365fd7fe593816f6b7bb2e7bba4b04e65fc45bf0e4a5576dba96df3bce, amount: 2000000000001 }
 o |   |   
---+---+---
   | x |   
---+---+---
   |   |   

additions:
Coin { parent_coin_info: 6b39e594a274d90c1d0504e987ed90c29a666fb611aef6bb653c8b669032574b, puzzle_hash: d55ecabeaaceed112d51f8e03e9837e52b4e3da12a508bf5d663e6e557079f8e, amount: 2000000000001 }
removals:
Coin { parent_coin_info: 79b7aaa318044a80ced7891fa6b8a0cc70e829ce73f04b8db11c29af7da50c67, puzzle_hash: 63d201d68ec302b4d15945c9a169fe397ab6aaf294b33f32c779f902dc295a14, amount: 2000000000001 }
 o |   |   
---+---+---
 x | x |   
---+---+---
   |   |   
